# Boilerplate

In [1]:
import requests
import uuid
from tqdm import tqdm

# CarMax Listings

In [2]:
def fetch_carmax_listings(zip_code, take=100):
    base_url = "https://www.carmax.com/cars/api/search/run"
    all_items = []
    seen_vins = set()
    
    headers = {
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'content-type': 'application/json',
        'referer': 'https://www.carmax.com/cars',
        'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
    }

    params = {
        'uri': '/cars',
        'skip': -take,
        'take': take,
        'zipCode': zip_code,
        'shipping': '-1',
        'sort': 'price-asc',
        'radius': '25',
        'visitorID': str(uuid.uuid4())
    }
    
    # Use a session to automatically handle cookies across requests
    session = requests.Session()
    session.headers.update(headers)
    
    pbar = tqdm(desc="Fetching CarMax listings", unit="items", dynamic_ncols=True)
    
    while True:
        params['skip'] += take
        
        response = session.get(base_url, params=params)
        response.raise_for_status()
        
        data = response.json()
        items = data.get('items', [])
        
        # Track unique listings by VIN
        for item in items:
            vin = item.get('vin')
            seen_vins.add(vin)
        
        all_items.extend(items)
        pbar.update(len(items))
        
        # Extract totalCount and testsGroups from response
        total_count = data.get('totalCount', 'N/A')
        tests_groups = data.get('testsGroups', 'N/A')
        
        pbar.set_postfix({
            "total": len(all_items),
            "unique": len(seen_vins),
            "page": params['skip'] // take + 1,
            "totalCount": total_count,
            "testsGroups": tests_groups
        })
        
        if len(items) < take:
            break
    
    pbar.close()
    return all_items


In [3]:
all_listings = fetch_carmax_listings('78753', take=24)

Fetching CarMax listings: 811items [05:28,  2.47items/s, total=811, unique=810, page=34, totalCount=811, testsGroups=BestMatchEdmundsTest:test]


In [4]:
all_listings[0]

{'stockNumber': 27579172,
 'vin': 'ML32A3HJ8FH047486',
 'year': 2015,
 'make': 'Mitsubishi',
 'model': 'Mirage',
 'body': '4D Hatchback',
 'trim': 'DE',
 'basePrice': 9998.0,
 'originalPrice': None,
 'hasPriceDrop': False,
 'mileage': 67882,
 'storeId': 7154,
 'storeName': 'Austin North',
 'storeCity': 'Austin',
 'state': 'Texas',
 'stateAbbreviation': 'TX',
 'distance': 0.0,
 'averageRating': 3.4210526315789473,
 'numberOfReviews': 19,
 'repairPalData': None,
 'isNewArrival': False,
 'isTransferable': False,
 'features': ['Power Locks',
  'AM/FM Stereo',
  'Power Windows',
  'Auxiliary Audio Input',
  'Air Conditioning',
  'CD Audio',
  'ABS Brakes',
  'Rear Defroster',
  'Power Mirrors',
  'Cloth Seats',
  'Traction Control',
  'Side Airbags',
  'Overhead Airbags',
  'Rear Spoiler'],
 'highlightedFeatures': 'Auxiliary Audio Input,Rear Spoiler,Overhead Airbags,Traction Control,Side Airbags,Air Conditioning',
 'highlights': ['lowMilesPerYear', 'fuelEfficient'],
 'lastMadeSaleableDate':

In [5]:
from collections import Counter

Counter([l["vin"] for l in all_listings])

Counter({'2T3W1RFV2LW098267': 2,
         'ML32A3HJ8FH047486': 1,
         '3FADP4EJ0KM105780': 1,
         '3VWD17AJ7FM225596': 1,
         '1FMCU0GX4GUA44019': 1,
         'KNDJP3A5XF7802275': 1,
         '1FMCU0G73FUA65359': 1,
         '3VW117AU9HM017235': 1,
         '3VWC17AU5GM514497': 1,
         '1C4NJCBA4FD104695': 1,
         'JA32U2FU5HU013736': 1,
         '3VW2K7AJ8FM202140': 1,
         '1FMCU0GX6GUB23238': 1,
         '1G1PC5SH3G7165993': 1,
         'KNDPN3AC4J7442850': 1,
         'JM1DE1KY7E0185103': 1,
         '5NPEB4AC7EH879314': 1,
         '1N4AL3AP5FN920653': 1,
         '3VW217AU4FM031240': 1,
         '3N1CN7AP7KL833743': 1,
         'JA4AZ3A34JJ003719': 1,
         'KL4CJBSB1FB163767': 1,
         '3N1AB7AP5FY338725': 1,
         'JF1GPAT67FH201653': 1,
         '3N1AB7AP1JY340884': 1,
         '1N4AL3APXFN322638': 1,
         '1FMCU9JXXFUC21510': 1,
         '3C4PDCGBXGT169158': 1,
         'ZACCJABB8HPE59880': 1,
         '3KPFL4A76JE261923': 1,
         '

In [6]:
from pprint import pprint

pprint([l for l in all_listings if l["vin"] == "2T3W1RFV2LW098267"])

[{'averageRating': 4.5,
  'basePrice': 25998.0,
  'bestFinanceDecision': None,
  'body': '4D Sport Utility',
  'cylinders': 4,
  'distance': 12.1,
  'driveTrain': 'Front Wheel Drive',
  'engineSize': '2.5L',
  'engineTorque': 184,
  'engineTorqueRpm': 5000,
  'engineType': 'Gas',
  'exteriorColor': 'Black',
  'featureScore': 6,
  'features': ['AM/FM Stereo',
               'Entune',
               'Traction Control',
               'Rear Defroster',
               'Automated Cruise Control',
               'Cloth Seats',
               'Power Mirrors',
               'Lane Departure Warning',
               'Overhead Airbags',
               'Power Seat(s)',
               'Android Auto',
               'Air Conditioning',
               'Power Windows',
               'Power Locks',
               'Apple CarPlay',
               'Auxiliary Audio Input',
               'Blind Spot Monitor',
               'Alloy Wheels',
               'Side Airbags',
               'Satellite Radio Re